In [ ]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


In [ ]:
env = gym.make('CartPole-v0')
print("State Space:", env.observation_space)
print("Action Space:", env.action_space)

done, state = False, env.reset()[0]
total_reward = 0
while not done:
    action = env.action_space.sample()
    state, reward, done, _ = env.step(action)
    total_reward += reward

print("Total reward obtained by random agent:", total_reward)



1. **State and Action Spaces**:
   - The **state space** in the `CartPole-v0` environment consists of continuous variables: the position and velocity of the cart, as well as the angle and angular velocity of the pole. These state variables describe the dynamics of the cart and the pole, which the agent must learn to balance.
   - The **action space** is discrete, consisting of two actions: moving the cart to the left or right. These actions are crucial for the agent to balance the pole and prevent it from falling.

2. **Random Agent Evaluation**:
   - A random agent was used to explore the environment and understand its reward structure. The agent randomly selects actions (left or right) at each step, with the goal of keeping the pole balanced.
   - The **total reward** obtained by the random agent is the sum of the rewards for each time step, reflecting how long the agent was able to keep the pole balanced before it fell.



In [ ]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return torch.softmax(self.fc2(x), dim=-1)


In [ ]:
def compute_returns(rewards, gamma, reward_to_go):
    returns = []
    if reward_to_go:
        for t in range(len(rewards)):
            Gt = sum([gamma ** (k - t) * rewards[k] for k in range(t, len(rewards))])
            returns.append(Gt)
    else:
        G = sum([gamma ** t * r for t, r in enumerate(rewards)])
        returns = [G] * len(rewards)
    return returns
def normalize_advantages(advantages):
    advantages = (advantages - np.mean(advantages)) / (np.std(advantages) + 1e-8)
    return advantages
def policy_gradient(env, policy, optimizer, gamma=0.99, reward_to_go=True, advantage_norm=True, num_iterations=1000, batch_size=500):
    average_returns = []

    for i in range(num_iterations):
        states, actions, advantages = [], [], []
        episode_rewards = []
        episode_returns = []
        state = env.reset()
        done = False

        while len(states) < batch_size:

            action_probs = policy(torch.tensor(state, dtype=torch.float32))
            action = np.random.choice(len(action_probs), p=action_probs.detach().numpy())
            next_state, reward, done, _ = env.step(action)

            states.append(state)
            actions.append(action)
            episode_rewards.append(reward)

            state = next_state
            if done:

                returns = compute_returns(episode_rewards, gamma, reward_to_go)
                baseline = np.mean(returns)
                episode_advantages = returns - baseline if advantage_norm else returns
                if advantage_norm:
                    episode_advantages = normalize_advantages(episode_advantages)

                advantages.extend(episode_advantages)
                episode_returns.append(sum(episode_rewards))

                episode_rewards = []
                state = env.reset()
                done = False


        states_tensor = torch.tensor(np.array(states), dtype=torch.float32)
        actions_tensor = torch.tensor(actions[:len(advantages)], dtype=torch.int64)
        advantages_tensor = torch.tensor(advantages, dtype=torch.float32)


        optimizer.zero_grad()
        log_probs = torch.log(policy(states_tensor).gather(1, actions_tensor.unsqueeze(1)).squeeze())
        loss = -torch.mean(log_probs * advantages_tensor)
        loss.backward()
        optimizer.step()


        avg_return = np.mean(episode_returns)
        average_returns.append(avg_return)
        print(f"Iteration {i + 1}/{num_iterations}, Average Return: {avg_return}")

    return average_returns


In [ ]:
env = gym.make("CartPole-v0")
policy = PolicyNetwork(env.observation_space.shape[0], hidden_dim=128, output_dim=env.action_space.n)
optimizer = optim.Adam(policy.parameters(), lr=0.01)

average_returns_no_rtgo_no_norm = policy_gradient(env, policy, optimizer, reward_to_go=False, advantage_norm=False, num_iterations=5000)
average_returns_no_rtgo_norm = policy_gradient(env, policy, optimizer, reward_to_go=False, advantage_norm=True, num_iterations=5000)
average_returns_rtgo_no_norm = policy_gradient(env, policy, optimizer, reward_to_go=True, advantage_norm=False, num_iterations=5000)
average_returns_rtgo_norm = policy_gradient(env, policy, optimizer, reward_to_go=True, advantage_norm=True, num_iterations=5000)


In [ ]:
plt.plot(average_returns_no_rtgo_no_norm, label="No Reward-to-Go, No Normalization")
plt.plot(average_returns_no_rtgo_norm, label="No Reward-to-Go & Normalization")
plt.plot(average_returns_rtgo_no_norm, label="Reward-to-Go, No Normalization")
plt.plot(average_returns_rtgo_norm, label="Reward-to-Go & Normalization")
plt.xlabel("Iterations")
plt.ylabel("Average Return")
plt.legend()
plt.title("Policy Gradient with Variance Reduction Techniques")
plt.show()


####  Observations on Variance Reduction Techniques

The four configurations tested with variance reduction techniques yielded the following conclusions:

1. **No Reward-to-Go, No Normalization**:
   - This baseline configuration experienced high variance in returns and slow convergence. Without normalization or reward-to-go, the learning process was unstable, showing fluctuations in the returns over time.

2. **No Reward-to-Go with Normalization**:
   - Adding advantage normalization stabilized the learning process by reducing the variance of the gradient updates, leading to smoother returns. Convergence improved, though it was still slower than configurations using reward-to-go.

3. **Reward-to-Go, No Normalization**:
   - The reward-to-go technique improved stability by reducing variance in gradient estimates, as it allowed rewards to reflect immediate action results rather than the entire trajectory. This configuration had better convergence than both configurations without reward-to-go.

4. **Reward-to-Go with Normalization**:
   - Combining reward-to-go with advantage normalization produced the most stable and efficient learning process. The returns converged smoothly with minimal variance, making this configuration the most effective for maximizing average return over iterations.

**Conclusion**:  
**Reward-to-Go with Advantage Normalization** yielded the most stable and efficient learning, making it the preferred configuration. Using both techniques together helped in significantly reducing gradient variance and improving policy convergence speed.

In [ ]:
batch_sizes = [200, 300, 400,500]
all_returns = {}

for batch_size in batch_sizes:
    print(f"\nRunning policy gradient with batch size {batch_size}")
    returns = policy_gradient(env, policy, optimizer, reward_to_go=True, advantage_norm=True, num_iterations=1000, batch_size=batch_size)
    all_returns[batch_size] = returns




In [ ]:
for batch_size, returns in all_returns.items():
    plt.plot(returns, label=f"Batch Size {batch_size}")

plt.xlabel("Iterations")
plt.ylabel("Average Return")
plt.legend()
plt.title("Impact of Batch Size on Policy Gradient Performance")
plt.show()

#### Impact of Batch Size on Policy Gradient Performance

Batch sizes of 200, 300, 400, and 500 were tested to analyze their impact on the performance of the policy gradient algorithm.

1. **Batch Size = 200**:
   - Provided quick updates and faster convergence, but with a high variance in returns due to smaller sample sizes per update. Learning was less stable, with noticeable fluctuations in performance.

2. **Batch Size = 300**:
   - Showed a good balance between stability and convergence speed. The returns were smoother, with reduced variance compared to batch size 200, and convergence was efficient.

3. **Batch Size = 400**:
   - Achieved greater stability with even smoother returns. Convergence was slower than batch sizes 200 and 300, but the stability made learning more reliable.

4. **Batch Size = 500**:
   - This batch size offered the most stable learning curve with minimal fluctuations, but convergence was slower due to less frequent updates. Stability was maximized, but the high batch size made the algorithm update less often, leading to slower overall learning.

**Conclusion**:  
**Batch sizes of 300 and 400** provided the best trade-offs, balancing stability and convergence speed. Larger batch sizes (400–500) increased stability but resulted in slower updates, while smaller batch sizes (200) converged faster but showed high variance.